In [31]:
import pymysql
import pandas as pd
from IPython.display import clear_output

pd.options.display.max_rows = 10

# SQL練習問題回答
[SQL練習問題 | TECH Projin][url]

[url]:http://tech.pjin.jp/blog/2016/12/05/sql%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C-%E4%B8%80%E8%A6%A7%E3%81%BE%E3%81%A8%E3%82%81/

## pandasで全部やるバージョン

In [30]:
connect = pymysql.connect(
    user='wcup2014',
    passwd='wcup2014',
    db='wcup2014',
    charset='utf8mb4'
)    

tables = pd.read_sql_query(
    "show tables",connect)

countries = pd.read_sql_query(
    'select * from countries;',
    conn
)

goals = pd.read_sql_query(
    'select * from goals;',
    conn
)

pairings = pd.read_sql_query(
    'select * from pairings;',
    conn
)

players = pd.read_sql_query(
    'select * from players;',
    conn
)

## 1. [各グループの中でFIFAランクが最も高い国と低い国のランキング番号を表示してください。](http://tech.pjin.jp/blog/2016/04/30/sql%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C-%E5%95%8F1/)

In [100]:
countries.groupby('group_name')['ranking'].agg([max, min]).reset_index()

,group_name,max,min
0,A,56,3
1,B,62,1
2,C,46,8
3,D,28,7
4,E,33,6
5,F,44,5
6,G,37,2
7,H,57,11


## 2. [全ゴールキーパーの平均身長、平均体重を表示してください。](http://tech.pjin.jp/blog/2016/04/30/sql%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C-%E5%95%8F2/)

In [107]:
players[players['position'] == 'GK'][['height','weight']].mean()

height    187.625000
weight     82.885417
dtype: float64

## 3. [各国の平均身長を高い方から順に表示してください。ただし、FROM句はcountriesテーブルとしてください。](http://tech.pjin.jp/blog/2016/04/30/sql%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C-%E5%95%8F3/)

In [132]:
pd.merge(
    players,
    countries.rename({'id':'country_id'}, axis=1),
    on='country_id',
    how='left',
).rename(
    {'name_y':'country'}, 
    axis=1,
)[
    ['country','height']
].groupby('country')['height'].mean().sort_values(ascending=False)

country
ドイツ             185.739130
ベルギー            185.173913
ボスニア・ヘルツェゴビナ    184.913043
ギリシャ            184.608696
韓国              183.826087
                   ...    
ホンジュラス          179.826087
エクアドル           178.826087
日本              178.521739
メキシコ            177.521739
チリ              176.217391
Name: height, Length: 32, dtype: float64

## 4. [各国の平均身長を高い方から順に表示してください。ただし、FROM句はplayersテーブルとして、テーブル結合を使わず副問合せを用いてください。](http://tech.pjin.jp/blog/2016/04/30/sql%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C-%E5%95%8F4/)

In [158]:
players.assign(
    country_name=(lambda df: df['country_id'].map(
        lambda i: countries[countries['id'] == i]['name'].values[0]
    ))
).groupby('country_name')['height'].mean().sort_values(ascending=False)

country_name
ドイツ             185.739130
ベルギー            185.173913
ボスニア・ヘルツェゴビナ    184.913043
ギリシャ            184.608696
韓国              183.826087
                   ...    
ホンジュラス          179.826087
エクアドル           178.826087
日本              178.521739
メキシコ            177.521739
チリ              176.217391
Name: height, Length: 32, dtype: float64

## 5. [キックオフ日時と対戦国の国名をキックオフ日時の早いものから順に表示してください。](http://tech.pjin.jp/blog/2016/05/12/sql%E7%B7%B4%E7%BF%92%E5%95%8F%E9%A1%8C-%E5%95%8F5/)

In [185]:
pairings.pipe(
    lambda df: pd.merge(
        df,
        countries.rename({'id':'my_country_id'}, axis=1),
        on='my_country_id',
        how='left',
    )
).pipe(
    lambda df:pd.merge(
        df,
        countries.rename({'id':'enemy_country_id'}, axis=1),
        on='enemy_country_id',
        how='left'
    )
)[['kickoff', 'name_x', 'name_y']].rename(
    {'name_x':'own', 'name_y':'enemy'},
    axis=1
).sort_values('kickoff')

,kickoff,own,enemy
0,2014-06-13 05:00:00,ブラジル,クロアチア
64,2014-06-13 05:00:00,クロアチア,ブラジル
1,2014-06-14 01:00:00,メキシコ,カメルーン
65,2014-06-14 01:00:00,カメルーン,メキシコ
2,2014-06-14 04:00:00,スペイン,オランダ
...,...,...,...
126,2014-07-13 05:00:00,ブラジル,オランダ
142,2014-07-13 05:00:00,オランダ,ブラジル
63,2014-07-14 04:00:00,アルゼンチン,ドイツ
127,2014-07-14 04:00:00,ドイツ,アルゼンチン
